In [89]:
import requests

tabla_url = "https://e2jaochskab4mlg3v5sb6uimuy0pnhov.lambda-url.us-east-1.on.aws/"
response = requests.request("GET", tabla_url)
list_ids=response.json()
ids=['<strong> Tabla de Posiciones:</strong>','']
for i,x in enumerate(list_ids):
    a=x['name']
    ids.append(str(i+1) + '{0: <35}'.format(a) + str(0))


In [92]:
ids=[' Tabla de Posiciones:','']
for i,x in enumerate(list_ids):
    a=x['name']
    ids.append(str(i+1) +' '+ a +(25-len(a))*' '+ str(0))

In [93]:
'\n'.join(ids)

' Tabla de Posiciones:\n\n1 fer                      0'

In [87]:

url = "https://api.telegram.org/bot5597623661:AAFyWZD91HiPIeaKmF89iIsxAp9lgmfF5Ig/sendMessage"

payload={'text': '```'+'\n'.join(ids)+'```',
'chat_id': '224798982',
'parse_mode': 'MarkdownV2'}

response = requests.request("POST", url,data=payload)



In [64]:
len('2 andres.armua@ed.ac.uk____0')

28

In [81]:
response

<Response [200]>

In [1]:
import requests

In [5]:
url='https://x14blnzsqf.execute-api.us-east-2.amazonaws.com/prod/fixture'

In [3]:
fix=requests.get(url)

In [4]:
fix


<Response [401]>

## TEST

In [46]:
import requests

headers = {
    'authority': 'x14blnzsqf.execute-api.us-east-2.amazonaws.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
    'dnt': '1',
    'origin': 'https://www.lanacion.com.ar',
    'referer': 'https://www.lanacion.com.ar/',
    'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
    'sec-ch-ua-mobile': '?1',
    'sec-ch-ua-platform': '"Android"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Mobile Safari/537.36',
    'x-token': '36FE17A0-2E7B-456F-9855-124601188D55',
    'x-value': 'f794a873-33e0-40d1-99d0-c06331a6d507',
}

response = requests.get('https://x14blnzsqf.execute-api.us-east-2.amazonaws.com/prod/fixture', headers=headers)

In [48]:
fix=response.json()

In [51]:
def partidosTerminados(fix):
    term=[]
    for x in fix:
        if x['status'] == 'full_time':
            term.append(x)
    return term

def partidosHoy(fix):
    hoy=[]
    for x in fix:
        dia=datetime.strptime(x['datetime'], '%Y-%m-%dT%H:%M:%S%z').date()
        if  dia == datetime.today().date():
            hoy.append(x)
    
    return hoy


In [52]:
from datetime import date

today = date.today()
today

datetime.date(2022, 11, 25)

In [55]:
from datetime import datetime
from zoneinfo import ZoneInfo

datetime.strptime(fix[0]['datetime'], '%Y-%m-%dT%H:%M:%S%z').astimezone(ZoneInfo('America/Buenos_Aires'))


datetime.datetime(2022, 11, 20, 13, 0, tzinfo=zoneinfo.ZoneInfo(key='America/Buenos_Aires'))

In [56]:
datetime.strptime(fix[0]['datetime'], '%Y-%m-%dT%H:%M:%S%z').astimezone(ZoneInfo('America/Buenos_Aires')).hour


13

In [57]:
datetime.strptime(fix[0]['datetime'], '%Y-%m-%dT%H:%M:%S%z').date() == datetime.today().date()


False

In [58]:
finished=partidosTerminados(fix)

In [60]:
import pandas as pd

In [61]:
df=pd.read_csv('results.csv')

In [64]:
import json

In [65]:
preds=json.loads(df.iloc[0].preds)

In [66]:
len(preds)

96

In [67]:
import numpy as np
apreds=np.array(preds)
preds2=apreds.reshape([8,12])

In [69]:
fpreds=[]
for x in preds2:
    fpreds.append(x.reshape([6,2]))

In [71]:
len(fix[:48])

48

In [72]:
def calculoPuntos(fix,n):
    print(df.name.iloc[n])
    preds=json.loads(df.preds.iloc[n])
    finished=partidosTerminados(fix)
    preds_dict=get_preds_dict(fix,preds)
    puntos=[]
    for x in finished:
        print('Resultado:',x['team_1'],x['goal_1_opta'],x['team_2'],x['goal_2_opta'])
        print('Prediccion:',x['team_1'],preds_dict[x['match_id']][0]['S'],x['team_2'],preds_dict[x['match_id']][1]['S'])

        resultado = (int(x['goal_1_opta']), int(x['goal_2_opta']))
        prediccion = (int(preds_dict[x['match_id']][0]['S']), int(preds_dict[x['match_id']][1]['S']))
        score=puntaje(prediccion, resultado)
        puntos.append(score)
        print('Obtuviste:',score ,'puntos')
        print()
    print('Total ',np.array(puntos).sum(),'Puntos.')

def calculoScore(fix,preds):
    finished=partidosTerminados(fix)
    preds_dict=get_preds_dict(fix,preds)
    puntos=[]
    for x in finished:
  
        resultado = (int(x['goal_1_opta']), int(x['goal_2_opta']))
        prediccion = (int(preds_dict[x['match_id']][0]['S']), int(preds_dict[x['match_id']][1]['S']))
        score=puntaje(prediccion, resultado)
        puntos.append(score)
    return np.array(puntos).sum()

In [73]:
def get_preds_dict(fix,preds):
    groups=['A','B','C','D','E','F','G','H']

    fixGrupos=[]
    for n in groups:
        grupo=[]
        for x in fix[:48]:
            if x['group']== n:
                grupo.append(x)
        fixGrupos.append(grupo)       

    ids_partidos=[]
    for i,x in enumerate(fixGrupos):
        for j,y in enumerate(x):
            if j == 4:
                ids_partidos.append(x[5]['match_id'])
            elif j == 5:
                ids_partidos.append(x[4]['match_id'])
            else:
                ids_partidos.append(y['match_id'])
    apreds=np.array(preds)
    preds48x2=apreds.reshape([48,2])
    preds_dict=dict(zip(ids_partidos, preds48x2))     
    return preds_dict

def puntaje(prediccion, resultado):
    if prediccion == resultado:
        return 5

    if np.sign(resultado[0] - resultado[1]) == np.sign(prediccion[0] - prediccion[1]):
        return 3

    return 0

# Armo la tabla

In [75]:
def tablapuntos(x):
    preds=json.loads(x)
    return calculoScore(fix,preds)

In [76]:
puntos=df.preds.apply(lambda x: tablapuntos(x))

In [77]:
df['puntos']=puntos

In [78]:
df.sort_values('puntos',ascending=False)

,id,name,preds,puntos
0,12525649_lgtrombetta@gmail.com,Leonardo,"[{""S"":""0""},{""S"":""2""},{""S"":""1""},{""S"":""3""},{""S"":...",39
4,720065757_ariel.haimovici@gmail.com,Ariel,"[{""S"":""0""},{""S"":""1""},{""S"":""0""},{""S"":""2""},{""S"":...",39
2,26915036_fede.barabas@gmail.com,Fede,"[{""S"":""0""},{""S"":""2""},{""S"":""1""},{""S"":""1""},{""S"":...",33
6,619906008_marianobarella@gmail.com,Mariano,"[{""S"":""1""},{""S"":""2""},{""S"":""1""},{""S"":""1""},{""S"":...",33
1,83230506_esonoparecefuncionar@gmail.com,Mer,"[{""S"":""1""},{""S"":""2""},{""S"":""1""},{""S"":""2""},{""S"":...",30
3,415250188_nahuel.freitas@gmail.com,freitas,"[{""S"":""1""},{""S"":""1""},{""S"":""0""},{""S"":""2""},{""S"":...",30
9,1277024606_gelpi640@gmail.com,Juli,"[{""S"":""1""},{""S"":""1""},{""S"":""0""},{""S"":""3""},{""S"":...",29
5,5206321186_ghisolfi_alina@hotmail.com,Alina,"[{""S"":""1""},{""S"":""0""},{""S"":""1""},{""S"":""2""},{""S"":...",27
7,630434327_andresarmua@gmail.com,Andres,"[{""S"":""1""},{""S"":""2""},{""S"":""0""},{""S"":""3""},{""S"":...",27
8,153626114_ionatan@gmail.com,Ionatan,"[{""S"":""1""},{""S"":""1""},{""S"":""1""},{""S"":""2""},{""S"":...",24


In [79]:
df2=df.sort_values('puntos',ascending=False)

In [84]:
df3=df2.drop(['id','preds'], axis=1)

In [ ]:
def get_tabla(df):
    puntos=df.preds.apply(lambda x: tablapuntos(x))
    df['puntos']=puntos
    df2=df.sort_values('puntos',ascending=False)
    return df2.drop(['id','preds'], axis=1)

In [98]:
df3

,name,puntos
0,Leonardo,39
4,Ariel,39
2,Fede,33
6,Mariano,33
1,Mer,30
3,freitas,30
9,Juli,29
5,Alina,27
7,Andres,27
8,Ionatan,24


In [99]:
for i,x in df3.iterrows():
    print(x['name'],x['puntos'])

Leonardo 39
Ariel 39
Fede 33
Mariano 33
Mer 30
freitas 30
Juli 29
Alina 27
Andres 27
Ionatan 24


In [85]:
df3.to_json()

'{"name":{"0":"Leonardo","4":"Ariel","2":"Fede","6":"Mariano","1":"Mer","3":"freitas","9":"Juli","5":"Alina","7":"Andres","8":"Ionatan"},"puntos":{"0":39,"4":39,"2":33,"6":33,"1":30,"3":30,"9":29,"5":27,"7":27,"8":24}}'

# Partidos Hoy

In [47]:
import requests
from datetime import datetime
from zoneinfo import ZoneInfo

headers = {
    'authority': 'x14blnzsqf.execute-api.us-east-2.amazonaws.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9,es;q=0.8',
    'dnt': '1',
    'origin': 'https://www.lanacion.com.ar',
    'referer': 'https://www.lanacion.com.ar/',
    'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
    'sec-ch-ua-mobile': '?1',
    'sec-ch-ua-platform': '"Android"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Mobile Safari/537.36',
    'x-token': '36FE17A0-2E7B-456F-9855-124601188D55',
    'x-value': 'f794a873-33e0-40d1-99d0-c06331a6d507',
}

response = requests.get('https://x14blnzsqf.execute-api.us-east-2.amazonaws.com/prod/fixture', headers=headers)
fix=response.json()

def partidosHoy(fix):
    hoy=[]
    for x in fix:
        dia=datetime.strptime(x['datetime'], '%Y-%m-%dT%H:%M:%S%z').date()
        if  dia == datetime.today().date():
            hoy.append(x)    
    return hoy

def printHoy():
    msg=[]
    hoy=partidosHoy(fix)
    for x in hoy:
        hora=datetime.strptime(x['datetime'], '%Y-%m-%dT%H:%M:%S%z').astimezone(ZoneInfo('America/Buenos_Aires')).hour
        msg.append(' '.join([x['team_1'],str(x['goal_1_opta']),' vs. ',x['team_2'],str(x['goal_2_opta']),'-',str(hora),'Hs.']))

    return msg

In [48]:
msg=printHoy()

In [49]:
print(msg)

['Gales 0  vs.  Irán 2 - 7 Hs.', 'Qatar None  vs.  Senegal None - 10 Hs.', 'Holanda None  vs.  Ecuador None - 13 Hs.', 'Inglaterra None  vs.  EE.UU. None - 16 Hs.']


In [50]:
r_url='https://cfrlqnbabuwcvij3npnelltley0wilsp.lambda-url.us-east-1.on.aws'
response=requests.request("GET", r_url)

In [51]:
response.json()

['Gales 0  vs.  Irán 2 - 7 Hs.',
 'Qatar None  vs.  Senegal None - 10 Hs.',
 'Holanda None  vs.  Ecuador None - 13 Hs.',
 'Inglaterra None  vs.  EE.UU. None - 16 Hs.']

# tabla

In [52]:
tb='https://e2jaochskab4mlg3v5sb6uimuy0pnhov.lambda-url.us-east-1.on.aws/'

In [53]:
response=requests.request("GET", tb)

In [56]:
response.text

'[{"puntos": 24, "name": "Ionatan"}, {"puntos": 33, "name": "Fede"}, {"puntos": 27, "name": "Alina"}, {"puntos": 39, "name": "Leonardo"}, {"puntos": 27, "name": "Andres"}, {"puntos": 39, "name": "Ariel"}, {"puntos": 30, "name": "Mer"}, {"puntos": 30, "name": "freitas"}, {"puntos": 29, "name": "Juli"}, {"puntos": 33, "name": "Mariano"}]'

In [57]:
import json


tabla_string = response.text
tabla = json.loads(tabla_string)

tabla = sorted(tabla, key=lambda x: x["puntos"], reverse=True)

tabla_string_ordered = str(tabla)

In [58]:
tabla_string_ordered

"[{'puntos': 39, 'name': 'Leonardo'}, {'puntos': 39, 'name': 'Ariel'}, {'puntos': 33, 'name': 'Fede'}, {'puntos': 33, 'name': 'Mariano'}, {'puntos': 30, 'name': 'Mer'}, {'puntos': 30, 'name': 'freitas'}, {'puntos': 29, 'name': 'Juli'}, {'puntos': 27, 'name': 'Alina'}, {'puntos': 27, 'name': 'Andres'}, {'puntos': 24, 'name': 'Ionatan'}]"

In [59]:
json.dumps(tabla_string_ordered)

'"[{\'puntos\': 39, \'name\': \'Leonardo\'}, {\'puntos\': 39, \'name\': \'Ariel\'}, {\'puntos\': 33, \'name\': \'Fede\'}, {\'puntos\': 33, \'name\': \'Mariano\'}, {\'puntos\': 30, \'name\': \'Mer\'}, {\'puntos\': 30, \'name\': \'freitas\'}, {\'puntos\': 29, \'name\': \'Juli\'}, {\'puntos\': 27, \'name\': \'Alina\'}, {\'puntos\': 27, \'name\': \'Andres\'}, {\'puntos\': 24, \'name\': \'Ionatan\'}]"'

In [61]:
tabla_url = "https://e2jaochskab4mlg3v5sb6uimuy0pnhov.lambda-url.us-east-1.on.aws/"
response = requests.request("GET", tabla_url)
list_ids=response.json()


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)

In [65]:
json.loads(response.text.replace("\'", "\""))

[{'puntos': 39, 'name': 'Leonardo'},
 {'puntos': 39, 'name': 'Ariel'},
 {'puntos': 33, 'name': 'Fede'},
 {'puntos': 33, 'name': 'Mariano'},
 {'puntos': 30, 'name': 'Mer'},
 {'puntos': 30, 'name': 'freitas'},
 {'puntos': 29, 'name': 'Juli'},
 {'puntos': 27, 'name': 'Alina'},
 {'puntos': 27, 'name': 'Andres'},
 {'puntos': 24, 'name': 'Ionatan'}]